In [ ]:
%pylab inline

import astropy
from astropy.io import fits
from astropy import table
from astropy.coordinates import Angle
from astropy import units as u

import galsim
import galsim.angle
import os
import descwl
import copy

from glob import glob
import os
import fitsio
import subprocess
from skimage.filters.rank import median

In [ ]:
## Make sure to change all of the relevant paths! 

In [ ]:
results = descwl.output.Reader('/Users/amandapagul/Desktop/DES_i_Amanda.fits').results

In [ ]:
mirror_diameter = 3.934
effective_area = 10.014
image_width = 215
image_height = 215
pixel_scale = 0.263
exposure_time = 800.
sky_brightness = 21.4
zenith_psf_fwhm = 1.03
zero_point = 15.65
extinction = 0.09

In [ ]:
dim = 128
def make_galaxy_snr(snr_level=500., sky_level=results.survey.mean_sky_level, rng=None,n=None,re=None,ba=None,pa=None):
    sky_level_pixel = sky_level / (results.survey.pixel_scale*dim)**2
    noise = galsim.PoissonNoise(rng, sky_level=sky_level_pixel)
    galaxy_flux = np.sqrt(sky_level)*snr_level
    gal = galsim.Sersic(n, half_light_radius=re/results.survey.pixel_scale, flux=galaxy_flux, flux_untruncated=False, trunc=16.)
    rad = gal.half_light_radius
    gal_shape = galsim.Shear(q=ba, beta=pa)
    gal = gal.shear(gal_shape)
    psf = galsim.Kolmogorov(fwhm=0.91, flux=1.0)
    final_gal = galsim.Convolve([psf, gal])
    image_gal = galsim.ImageF(dim, dim)
    final_gal.drawImage(image_gal)
    #bright_galaxy_copy = gal.withFlux(galaxy_flux)
    #final = galsim.Convolve([psf, bright_galaxy_copy])
    #image = galsim.ImageF(dim, dim)
    #image = image.withFlux(galaxy_flux)
    #final.drawImage(image)
    image_copy = image_gal.copy()
    image_copy.addNoise(noise)
    noise = image_copy.array-image_gal.array
    #print re, gal.half_light_radius, gal.scale_radius, gal.flux, gal.flux/np.sqrt(sky_level)
    return image_gal.array, image_copy.array, noise, galaxy_flux, gal.flux, rad

random_seed = 123456
rng = galsim.BaseDeviate(random_seed)
 
for i in range(0,1000):
    try:
        n = np.random.rand(1)*(6.2-0.3)+0.3
        re = (results.table['a'][i]**2+results.table['b'][i]**2)**0.5
        ba = results.table['b'][i]/results.table['a'][i]
        pa = results.table['beta'][i]*galsim.radians
        #gal = galsim.Sersic(n, scale_radius=re, flux=results.table['flux'][i], flux_untruncated=True)
        #gal_shape = galsim.Shear(q=results.table['b'][i]/results.table['a'][i], beta=results.table['beta'][i]*galsim.radians)
        #gal = gal.shear(gal_shape)
        #psf = galsim.Kolmogorov(fwhm=0.91, flux=1.0)
        #final = galsim.Convolve([psf, gal])

        #image = galsim.ImageF(dim, dim)
        #final.drawImage(image)
        
        #image_epsf = galsim.ImageF(dim, dim)
        #psf.drawImage(image_epsf)
        #plt.imshow(image.array)


        snr_array = [5.,10.,20.,50.,100.]
        for snr_level in snr_array:
            #bright_galaxy_cp = image.copy()
            image1 = make_galaxy_snr(snr_level=snr_level,rng=rng,n=n,re=re,ba=ba,pa=pa)
            image2 = make_galaxy_snr(snr_level=snr_level,rng=rng,n=n,re=re,ba=ba,pa=pa)
            image3 = make_galaxy_snr(snr_level=snr_level,rng=rng,n=n,re=re,ba=ba,pa=pa)
            image4 = make_galaxy_snr(snr_level=snr_level,rng=rng,n=n,re=re,ba=ba,pa=pa)
            galnoise = [image1[1],image2[1],image3[1],image4[1]]
            noise = [image1[2],image2[2],image3[2],image4[2]]
            galmod = [image1[0]]


            prim_hdu = fits.PrimaryHDU(data=galnoise)
            noise_imgs = fits.ImageHDU(data=noise)
            galaxy_imgs = fits.ImageHDU(data=galmod)
            binary_table_hdu = fits.BinTableHDU(astropy.table.Table(results.table[i]))
            prim_hdu.header['SNRLEVEL'] = snr_level
            prim_hdu.header['X'] = dim/2.0#0.5*(results.table['xmax'][i]-results.table['xmin'][i])
            prim_hdu.header['Y'] = dim/2.0#0.5*(results.table['ymax'][i]-results.table['ymin'][i])
            prim_hdu.header['B/A'] = results.table['b'][i]/results.table['a'][i]
            prim_hdu.header['PA'] = np.degrees(results.table['beta'][i])
            prim_hdu.header['N'] = n[0]
            prim_hdu.header['Flux_in'] = image1[3]
            prim_hdu.header['Flux_out'] = image1[4]
            prim_hdu.header['ID'] = results.table['db_id'][i]
            prim_hdu.header['Re_in'] = re
            prim_hdu.header['Re_out'] = image1[5]
            hdulist = fits.HDUList([prim_hdu, noise_imgs, galaxy_imgs, binary_table_hdu])
            hdulist.writeto('./sims/galaxy_n%d_snr%.1f_id%d_%ix%i.fits.gz' % (4, snr_level, i, dim, dim), overwrite=True)   
            
    except Exception as e: print e

        

In [ ]:
galfit_header = ('''
# IMAGE and GALFIT CONTROL PARAMETERS
A) %s            # Input data image (FITS file)
B) %s            # Output data image block
C) none          # Sigma image name (made from data if blank or "none") 
D) /Users/amandapagul/Desktop/HFFCode/sims/psf.fits  #kolmogorov, fwhm=0.91, flux=1          # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) none                # Bad pixel mask (FITS image or ASCII coord list)
G) none                # File with parameter constraints (ASCII file) 
H) 1    128   1    128 # Image region to fit (xmin xmax ymin ymax)
I) 100    100          # Size of the convolution box (x y)
J) 13.94               # Magnitude photometric zeropoint 
K) 0.263  0.263        # Plate scale (dx dy)   [arcsec per pixel]
O) regular             # Display type (regular, curses, both)
P) 0                   # Options: 0=normal run; 1,2=make model/imgblock & quit


# THE OBJECT LIST BELOW can be however long or short as the complexity
# requires.  The user has complete freedom to mix and match the components
# by duplicating each object block.

# INITIAL FITTING PARAMETERS
#
# column 1:  Parameter number
# column 2: 
#          -- Parameter 0: the allowed functions are: sersic, nuker, expdisk
#	      edgedisk, devauc, king, moffat, gaussian, ferrer, psf, sky
#	   -- Parameter 1-10: value of the initial parameters
#          -- Parameter C0: For diskiness/boxiness
#             <0 = disky
#             >0 = boxy
#          -- Parameter Z: Outputting image options, the options are:
#             0 = normal, i.e. subtract final model from the data to create
#		  the residual image
#	      1 = Leave in the model -- do not subtract from the data
#
# column 3: allow parameter to vary (yes = 1, no = 0)
# column 4: comment

# sky

 0) sky
 1) 0.00       1       # sky background       [ADU counts]
 2) 0.000      0       # dsky/dx (sky gradient in x) 
 3) 0.000      0       # dsky/dy (sky gradient in y) 
 Z) 0                  #  Skip this model in output image?  (yes=1, no=0)
 
''')

sersic_function = '''
# Sersic function

 0) sersic             # Object type
 1) %f  %f 0 0    # position x, y        [pixel]
 3) %f      1       # total magnitude    
 4) %f      1       #     R_e              [Pixels]
 5) %f       1       # Sersic exponent (deVauc=4, expdisk=1)  
 9) %f       1       # axis ratio (b/a)   
10) %f       1       # position angle (PA)  [Degrees: Up=0, Left=90]
 Z) 0                  #  Skip this model in output image?  (yes=1, no=0)

'''


In [ ]:
from skimage.filters.rank import median
import os
files = glob('/Users/amandapagul/Desktop/HFFCode/sims/noiseless/*noiselessimg.fits')
start = time.time()
imgtype = 'noiseless'
os.chdir('/Users/amandapagul/Desktop/HFFCode/sims/noiseless/')
for filename in np.sort(files):    
  
    galfit_filename = filename.split(".fits")[0]
    
    #extract photometry from cutouts
    subprocess.check_call(['sex',str(filename),'-c','default.sex'])
    
    results = astropy.table.Table.read('/Users/amandapagul/Desktop/HFFCode/sims/noiseless/test.cat',format='ascii')
    x = results['X_IMAGE']
    y = results['Y_IMAGE']
    flux = results['FLUX_AUTO']
    kron = results['KRON_RADIUS']
    a = results['A_IMAGE']
    b = results['B_IMAGE']
    theta = results['THETA_IMAGE']
    flux_radius = results['FLUX_RADIUS']
    
    with open(galfit_filename+'.feedme', "w") as f: 
        f.write(galfit_header %(galfit_filename+".fits", galfit_filename+'_out.fits'))
        for i,obj in enumerate((results)):
            mag = -2.5*np.log10(flux[i])+13.94
            re = np.sqrt(a[i]**2+b[i]**2)
            ba = b[i]/a[i]
            pa = theta[i]+90.
            n = flux_radius[i]/kron[i]
            f.write(sersic_function %(x[i], y[i], mag, re, n, ba, pa))
    
    galfit_file = galfit_filename+'.feedme'
    subprocess.check_call(['/Users/amandapagul/Software/galfit',galfit_file])
    